In [1]:
%pip install --upgrade torch
%pip install transformers[torch]
%pip install --upgrade mlflow
%pip install --upgrade transformers
%pip install --upgrade datasets==2.9.0

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install evaluate

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install azureml-mlflow==1.52.0

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os 
import mlflow
import pandas as pd
import numpy as np
from datasets import load_dataset
import evaluate
from azureml.core import Workspace
from transformers import AutoModelForSequenceClassification,AutoTokenizer,TrainingArguments,Trainer
import pickle
import time
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential,AzureCliCredential

subscription_id = '80c77c76-74ba-4c8c-8229-4c3b2957990c'
resource_group = 'sonata-test-rg'
workspace_name = 'sonata-test-ws'

credential = AzureCliCredential()
ws = Workspace(subscription_id, resource_group, workspace_name)

workspace_ml_client = MLClient(
        credential, subscription_id, resource_group, ws
    )

# store the mlflow results in azure ml workspace
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-14 14:55:57.653590: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-14 14:55:58.580269: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
EXPERIMENT_NAME = "BertBaseUncased"
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='', creation_time=1689346486107, experiment_id='450a5349-b926-4a0a-9b61-9c7eb40881ec', last_update_time=None, lifecycle_stage='active', name='BertBaseUncased', tags={}>

In [6]:
%pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [7]:
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import (
    BertForSequenceClassification,
    BertTokenizerFast,
    BertModel,
    TrainingArguments,
    Trainer,
    EvalPrediction,
)

checkpoint = "bert-base-uncased"
model = BertForSequenceClassification.from_pretrained(checkpoint)
tokenizer = BertTokenizerFast.from_pretrained(checkpoint)
raw_ds= load_dataset("glue", "mrpc")
raw_ds['train'] = raw_ds['train'].shuffle().select(range(100))
raw_ds['test']= raw_ds['test'].shuffle().select(range(20))
raw_ds['validation']= raw_ds['validation'].shuffle().select(range(20))
metric = evaluate.load("glue", "mrpc")
dataset = raw_ds.map(
    lambda x: tokenizer(x["sentence1"], x["sentence2"], truncation=True),
    batched=True,
)
dataset = dataset.remove_columns(["sentence1", "sentence2", "idx"])
dataset = dataset.rename_column("label", "labels")
dataset = dataset.with_format("torch")

trainer_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")


def compute_metrics(eval_preds: EvalPrediction):
    x, y = eval_preds
    preds = np.argmax(x, -1)
    return metric.compute(predictions=preds, references=y)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
import mlflow.pytorch

In [9]:
mlflow.transformers.save_model(
        transformers_model={"model": model, "tokenizer" : tokenizer},
        path="./bert-base-uncasedsavemodel",
        # task="text-classification",
        # input_example="A clever and witty question",
    )

/tmp/ipykernel_9292/3585814701.py:1: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.29.2``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  mlflow.transformers.save_model(
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [10]:
print(mlflow.__version__)

2.4.2


In [11]:
%pip install torchvision

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [7]:
# test_dataset = load_dataset("glue","mrpc", split="test").shuffle().select(range(1000))
# train_dataset = load_dataset("glue", "mrpc", split="train").shuffle().select(range(1000))

Found cached dataset glue (/home/azureuser/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Found cached dataset glue (/home/azureuser/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [11]:
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# tokenizer.save_pretrained("./model/tokenizer")
# model = AutoTokenizer.from_pretrained("bert-base-uncased")
# model.save_pretrained("./model/weights")
# config = AutoTokenizer.from_pretrained("bert-base-uncased")
# config.save_pretrained("./model/config")

('./model/config/tokenizer_config.json',
 './model/config/special_tokens_map.json',
 './model/config/vocab.txt',
 './model/config/added_tokens.json',
 './model/config/tokenizer.json')

In [15]:
# model_path

'https://ml.azure.com/model/azureml_e3e0f964-1334-4f88-8edb-28288f793aca_output_mlflow_log_model_249175241:1/details?wsid=/subscriptions/80c77c76-74ba-4c8c-8229-4c3b2957990c/resourceGroups/sonata-test-rg/providers/Microsoft.MachineLearningServices/workspaces/sonata-test-ws&tid=72f988bf-86f1-41af-91ab-2d7cd011db47'

In [5]:
# %pip install --upgrade azureml-mlflow


Note: you may need to restart the kernel to use updated packages.


In [1]:
# %pip install azureml-mlflow==1.52.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
# import azureml.mlflow

# version = azureml.mlflow.__version__
# print("AzureML-MLflow version:", version)


AzureML-MLflow version: 1.52.0


In [7]:
# import mlflow

# version = mlflow.version.VERSION
# print("MLflow version:", version)


MLflow version: 2.4.2


In [12]:
with mlflow.start_run():
    # registered_model_name="Bert"
    model_local_path = os.path.abspath("./bert-base-uncasedsavemodel")
    mlflow.register_model(f"file://{model_local_path}", "BertModel")

Successfully registered model 'BertModel'.
2023/07/14 15:30:40 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: BertModel, version 1
Created version '1' of model 'BertModel'.


In [13]:
mlmodel = mlflow.transformers.load_model("bert-base-uncasedsavemodel")

/tmp/ipykernel_9292/216228871.py:1: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.29.2``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  mlmodel = mlflow.transformers.load_model("bert-base-uncasedsavemodel")
2023/07/14 15:33:46 WARNING mlflow.transformers: Could not specify device parameter for this pipeline type


In [14]:
mlmodel.model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [15]:
%pip install mlflow

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [16]:
trainer = Trainer(
    model=mlmodel.model,
    tokenizer=tokenizer,
    args=trainer_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"]
    # test_dataset=dataset["test"]
)

result = trainer.train()

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.590509,0.700000,0.812500
2,No log,0.647598,0.600000,0.692308
3,No log,0.675245,0.600000,0.692308


In [17]:
mlflow.end_run()